In [1]:
!pip install unidecode

In [2]:
import warnings
import requests as req
import numpy as np
import pandas as pd
import os
import json

from bs4 import BeautifulSoup
from unidecode import unidecode

warnings.filterwarnings('ignore')

## Constants definitions

In [3]:
URL = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos'
JSON_DIR = 'speechs'

## Functions

In [4]:
def get_url(url):
    response = req.get(url)
    if response.status_code != 200:
        print(f'Error {response.status_code} fetching {url}')

    return BeautifulSoup(response.text)

def create_savedir(dir):
    try:
        os.makedirs(dir)
        print('Directory created successfully')
    except OSError:
        print('Directory already exists')


## Classes

In [5]:
class WebPage():

    def __init__(self, url):
        self.url = url
        self.max_page = self.number_of_pages()
        self.pages_list = self.pages_list()
        
    def number_of_pages(self):
        bs = get_url(self.url)
        pagination = bs.find(class_='paginacao listingBar')
        max_page = np.array([int(page_index.text) 
                             for page_index in pagination.find_all('a', href=True) 
                             if page_index.text.isdecimal()]).max()
        return int(max_page)

    def pages_list(self):
        pages_list = []
        prefix = '?b_start:int='
        [pages_list.append(self.url + prefix + str(idx * 30)) for idx in range(self.max_page)]
        return pages_list
    
    def save_articles(self):
        for page_url in self.pages_list:
            print(f'Saving articles from {page_url}')
            bs = get_url(page_url)

            articles = []
            for article in bs.find_all('article'):
                article = Article(article)
                article.save_to_json(JSON_DIR)

        print('Articles saved sucessfully')

In [6]:
class Article():
    
    def __init__(self, article=None):
        
        if article is not None:
            self.link = article.find(class_='summary url')['href']

            datetime = article.find_all(class_='summary-view-icon')
            date = datetime[0].text.strip()
            self.date = "".join(reversed(date.split('/')))

            self.time = datetime[1].text.strip()

            content = get_url(self.link).find(id='parent-fieldname-text').text.strip()
            self.content = unidecode(content).replace('\n','')
    
    def __str__(self):
        return f'Article link:{self.link}, date:{self.date}, time:{self.time}, content:{self.content}'

    def save_to_json(self, folder):
        path = os.path.join(folder, f'{self.date}-{self.time}.json')
        with open(path, 'w') as file:
            json.dump(self.__dict__, file)

    def load_from_json(folder):
        files = os.listdir(folder)
        
        articles = []
        for file in files:
            path = os.path.join(folder, file)
            with open(path, 'r') as file:
                data = json.load(file)
                article = Article()
                article.link = data['link']
                article.date = data['date']
                article.time = data['time']
                article.content = data['content']
                articles.append(article.__dict__)
        print(f'{len(articles)} articles loaded sucessfully')
        return articles
                

## Load and save process

In [7]:
create_savedir(JSON_DIR)

Directory created successfully


In [8]:
webpage = WebPage(URL)
webpage.save_articles()

Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=0
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=30
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=60
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=90
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=120
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=150
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=180
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=210
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=240
Saving articles from https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b

## Load process

In [9]:
articles = Article.load_from_json(JSON_DIR)

361 articles loaded sucessfully


In [10]:
df = pd.DataFrame(articles)
df

,link,date,time,content
0,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20190101,15h10,"Congresso Nacional, 01 de janeiro de 2019 Exce..."
1,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20190101,16h45,"Palacio do Planalto, 01 de janeiro de 2018 ..."
2,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20190102,17h21,"Brasilia-DF, 02 de janeiro de 2019 ..."
3,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20190107,12h20,"Brasilia-DF, 07 de janeiro de 2019 Senhor vice..."
4,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20190115,12h10,"Brasilia/DF, 15 de janeiro de 2019 Como o povo..."
...,...,...,...,...
356,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20210322,12h32,"Palacio do Planalto, 22 de marco de 2021 Bom ..."
357,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20210322,19h14,"Palacio do Planalto, 22 de marco de 2021Boa ta..."
358,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20210325,17h46,"Palacio do Planalto, 25 de marco de 2021 Boa t..."
359,https://www.gov.br/planalto/pt-br/acompanhe-o-...,20210326,11h08,Bom dia.E com satisfacao que tomo parte nesta ...


In [11]:
content = df['content'][0]
content

'Congresso Nacional, 01 de janeiro de 2019 Excelentissimo presidente do Congresso Nacional, senador Eunicio Oliveira, Senhoras e senhores chefes de Estado, chefes de Governo, vice-chefes de Estado e vice-chefes de Governo, que me honram com suas presencas. Vice-presidente da Republica Federativa do Brasil, Hamilton Mourao, meu contemporaneo de Academia Militar de Agulhas Negras, Presidente da Camara dos Deputados, prezado amigo e companheiro, deputado Rodrigo Maia, Ex-presidentes da Republica Federativa do Brasil, senhor Jose Sarney, senhor Fernando Collor de Mello, Presidente do Supremo Tribunal Federal, ministro Dias Toffoli, Senhoras e senhores ministros de Estado e comandantes das Forcas aqui presentes, Procuradora-Geral da Republica, Raquel Dodge,Senhoras e senhores governadores,Senhoras e senhores senadores e deputados federais,Senhoras e senhores chefes de missoes estrangeiras acreditados junto ao governo brasileiro,Minha querida esposa Michelle, daqui vizinha Ceilandia, Meus fi